# Beautiful Soup

## Introdução

Nesta aula veremos como utilizar na prática a biblioteca BeautifulSoup, muito útil para realizar web scraping em sites que não utilizam conteúdos dinâmicos.

Este laboratório tem como base a própria documentação da bilioteca, que pode ser encontrada em:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

## Preâmbulo

Antes de rodar o código acima, é necessário instalar a biblioteca Beautiful Soup no seu ambiente de desenvolvimento Python.

É interessante criar um ambiente vitual de desenvolvimento antes de instalar a biblioteca. Neste curso, usaremos o conda para criar e gerenciar nossos ambientes virtuais.

Seguem os comandos para criar um abiente virtual e instalar a biblioteca no Python:

```
conda create --name ambiente-cpa-p3 python 3.12

conda activate ambiente-cpa-p3

conda install bs4 html5lib
```

---

## Obtendo páginas na Web

In [1]:
# Essa função é utilizada para recuperar o html de uma página web
from urllib.request import urlopen

In [2]:
# Abrindo uma página
html = urlopen('http://www.pythonscraping.com/pages/page1.html')
site = html.read()
print(site)

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


## Tratamento de Erros

Ao abrir uma url usano a função ``urlopen`` temos dois problemas principais que podem ocorrer:
- A página pode não estar no servidor
- O servidor não existe / não foi encontrado
No primeiro caso o servidor retornará um erro do tipo 404 - Page not found ou 500 - Internal server error. Em ambos os casos o python lança uma exceção do tipo ``HTTPError``. No segundo caso o erro que ocorre é ``URLError``.

Essas exceções devem sempre ser tratadas em nosso código, evitando erros desnecessários

In [3]:
from urllib.error import HTTPError, URLError

def get_pagina(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("Houve um erro na obtenção da página! ", e)
        return None
    except URLError as e:
        print("Ocorreu um erro no servidor!", e)
        return None
    else:
        print("Consegui abrir a página")
        return html.read()
    

In [4]:
pagina = get_pagina('http://www.google.com.br/lucas')
if pagina is not None:
    print(pagina)


Houve um erro na obtenção da página!  HTTP Error 404: Not Found


In [5]:
pagina = get_pagina('http://www.joogle.com.br/')
if pagina is not None:
    print(pagina)

KeyboardInterrupt: 

In [6]:
pagina = get_pagina('https://www.google.com.br')
if pagina is not None:
    print(pagina)

Consegui abrir a página
b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="pt-BR"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="jAn5vq4O3z7OUmIw33j9rQ">(function(){var _g={kEI:\'nbKjaI_BKM3A5OUPkqLMEQ\',kEXPI:\'0,202854,2,3497431,1097,448528,90133,94918,78219,266577,290044,5230280,11401,364,32768570,4043709,25228681,51190,5,66111,35070,4566,60606,6755,23878,7034,2106,4599,328,6225,64164,15049,8204,3296,4135,30376,28335,10902,12054,6290,19657,5307,353,4100,14780,5870,3858,3856,5773,15029,12583,4719,11805,6251,35,3420,5538,7946,12107,5683,3605,1083,16688,2975,16305,4219,6,3,5746,3,1187,4157,1738,3619,1215,1,3467,2,1554,4,1,2,2,2,2535,1338,727,1033,1579,5840,5742,3018,2646,99,2,2,4,1,320,781,4322,667,12287,730,3443,1588,1390,1600,4975,3563,2490,1280,407,183,4,1378,32,104,3044,27,281,540,69,450,2844,46,357,1

---

## Arquivo ``robots.txt``

In [7]:
import urllib.robotparser

In [8]:
# Lê o arquivo robots.txt
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://g1.globo.com/robots.txt')
rp.read()

In [9]:
rrate = rp.can_fetch("*", "https://g1.globo.com/")
print(rrate)

True


In [58]:
rrate = rp.can_fetch("GPTBot", "https://g1.globo.com/")
print(rrate)

False


In [59]:
rrate = rp.can_fetch("*", "https://g1.globo.com/jornalismo/g1/")
print(rrate)

False


---

## Basicão do BS

In [12]:
# Importando a classe principal da biblioteca BeautifulSoup
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'bs4'

In [8]:
# Cria o objeto BeautifulSoup
bs = BeautifulSoup(site, 'html.parser')
print(bs.prettify())

<html>
 <head>
  <title>
   A Useful Page
  </title>
 </head>
 <body>
  <h1>
   An Interesting Title
  </h1>
  <div>
   Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
  </div>
 </body>
</html>



In [9]:
print(bs.h1)

<h1>An Interesting Title</h1>


In [10]:
print(bs.h1.text)

An Interesting Title


Note que essa forma de indexar os elementos do HTML só permite retornar a **primeira ocorrência** de cada um. Por exemplo, se houver duas tagas `h1` em uma página somente a primeira poderá ser obtida dessa forma.

### Exercício
Recupere o conteúdo da tag `div`

---

Abaixo criaremos uma variável com o HTML que utilizaremos como exemplo neste laboratório. É um trecho de "Alice no País das Maravilhas" formatado em HTML.

In [11]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [12]:
soup = BeautifulSoup(html_doc)
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



## Acesso aos atributos e Movimentação

Podemos acessar os atributos de forma semelhante com a qual acessamos os valores de um dicionário.

In [13]:
soup.body.a['href']

'http://example.com/elsie'

### Exercício
Escreva o código que acessa o conteúdo do atributo class da primeira tag "p" que está dentro de body
A saída deve ser: `['title']`


['title']

Para acessar todos os filhos de uma tag, podemos utilizar o método ```.contents``` ou o gerador de listas ```.children```

In [14]:
tag_head = soup.head
tag_head

<head><title>The Dormouse's story</title></head>

In [15]:
tag_head.contents

[<title>The Dormouse's story</title>]

Em alguns casos podemos querer também os “filhos dos filhos”, nesse caso podemos utilizar o método ```.descendants```

In [16]:
for filho in tag_head.descendants:
    print(filho)

<title>The Dormouse's story</title>
The Dormouse's story


Quando a tag possui apenas uma ```NavigableString``` como filho, podemos acessar pelo ```.string```, caso possua mais de um podemos acessar via ```.strings``` e ```.stripped_strings```


In [17]:
tag_titulo = soup.head.title
tag_titulo

<title>The Dormouse's story</title>

In [18]:
tag_titulo.string

"The Dormouse's story"

In [19]:
print(type(tag_titulo.string))
titulo = str(tag_titulo.string)
print(titulo, type(titulo))

<class 'bs4.element.NavigableString'>
The Dormouse's story <class 'str'>


Podemos visitar também tags irmãs acessando os métodos ```.next_siblings``` e ```.previous_siblings```

In [20]:
link = soup.a
print(link)

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>


In [21]:
link.next_sibling

',\n'

In [22]:
for irmao in link.next_siblings:
    print("[", irmao, "]")

[ ,
 ]
[ <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> ]
[  and
 ]
[ <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a> ]
[ ;
and they lived at the bottom of a well. ]


## Funções de Busca

As funções de busca tem como objetivo encontrar elementos dentro das páginas web. Existem duas funções de busca: `find` e `find_all`

### Busca por string

In [23]:
soup.find('a')

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [24]:
soup.find_all('a')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### Exercício
Qual a diferença entre as funções find e find_all?

### Busca por Regex

In [25]:
import re 
for tag in soup.find_all(re.compile("^b")):
    print(tag.name)

body
b


### Busca por lista
Match com um elemento de uma lista

In [26]:
soup.find_all(["a", "b"])

[<b>The Dormouse's story</b>,
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### True

In [27]:
for tag in soup.find_all(True):
    print(tag.name)

html
head
title
body
p
b
p
a
a
a
p


### Função

Match com elementos que retornam True quando passados para a função

In [28]:
def has_class_but_no_id(tag):
   return tag.has_attr('class') and not tag.has_attr('id')

for tag in soup.find_all(has_class_but_no_id):
   print(tag)
   print()

<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>



In [29]:
from bs4 import NavigableString
def surrounded_by_strings(tag):
    return (isinstance(tag.next_element, NavigableString)
            and isinstance(tag.previous_element, NavigableString))

for tag in soup.find_all(surrounded_by_strings):
    print(tag.name)   

body
p
a
a
a
p


### Parâmetros da função find_all

```
find_all(name, attrs, recursive, string, limit, **kwargs)
```

```name```: para filtrar apenas tags com o nome específico

```attrs```: utilizado para realizar filtros de atributos

```recursive```: pesquisar apenas na tag ou em seus descendentes

```string```: pesquisar pelo conteúdo/string das tags

```limit```: limita o número de retornos do ```find_all```

```**kwargs```: Todos os parâmetros nomeados não conhecidos são convertidos para filtros de atributos

In [30]:
# Parâmetro nome
soup.find_all("title")

[<title>The Dormouse's story</title>]

In [31]:
# Parâmetro attrs
soup.find_all(attrs={"class": "sister"})

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [32]:
# Atributo recursive
soup.html.find_all("title")

[<title>The Dormouse's story</title>]

In [33]:
# Esse código só busca nos filhos diretos da tag html
soup.html.find_all("title", recursive=False)

[]

In [34]:
# Atributo string
soup.find_all(string="Elsie")

['Elsie']

In [35]:
soup.find_all(string=["Tillie", "Lacie", "Elsie"])

['Elsie', 'Lacie', 'Tillie']

In [36]:
import re
soup.find_all(string=re.compile("Dormouse"))

["The Dormouse's story", "The Dormouse's story"]

In [37]:
def is_the_only_string_within_a_tag(s):
    """Retorna True se a string for o filho único da tag pai."""
    return (s == s.parent.string)

soup.find_all(string=is_the_only_string_within_a_tag)

["The Dormouse's story",
 "The Dormouse's story",
 'Elsie',
 'Lacie',
 'Tillie',
 '...']

In [38]:
# Parâmetro limit
soup.find_all("a", limit=2)

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [39]:
# Parâmetro **kwargs
soup.find_all(id="link2")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [40]:
soup.find_all(href=re.compile("elsie"))

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

In [41]:
soup.find_all(class_="sister")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### Exercício

Usando ```find_all``` selecione:
1. Todas as tags da classe story
2. Tag com id link2 e as suas irmãs subsequentes 


## Seletores CSS

Seletores são padrões textuais que casam com algum elemento de uma árvore (de um arquivo XML ou HTML). 

Podemos ter os seguintes tipos de seletores:

* **Seletores Simples**: Por tipo, universal, por id, por classe, por atributo ou pseudo-classe;
* **Seletores Compostos**: É uma sequência de seletores simples (por exemplo por tipo e por classe);
* **Lista de Seletores**: É formada por uma lista de seletores separadas por vírgula
* **Seletores Complexos**: É formado por múltiplos seletores simples ou compostos junto de combinadores.


In [42]:
# Seletor por tipo
soup.select("title")

[<title>The Dormouse's story</title>]

In [43]:
# Seletor Universal
soup.select("*")

[<html><head><title>The Dormouse's story</title></head>
 <body>
 <p class="title"><b>The Dormouse's story</b></p>
 
 <p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>
 
 <p class="story">...</p>
 </body></html>,
 <head><title>The Dormouse's story</title></head>,
 <title>The Dormouse's story</title>,
 <body>
 <p class="title"><b>The Dormouse's story</b></p>
 
 <p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at 

In [44]:
# Seletor por atributo
soup.select("[id]")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [45]:
# Seletor de atributo por substring
soup.select("[class^=s]")

[<p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>,
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>,
 <p class="story">...</p>]

In [46]:
# Seletor de atributo por classe
soup.select(".sister")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [47]:
# Seletor de Id
soup.select("#link2")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [48]:
# Seletores Compostos
soup.select("a[href$=tillie]")

[<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### Seletores com Combinadores

In [49]:
# Combinador Espaço ' '
soup.select("body a")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [50]:
# Combinador Filho >
soup.select("head > title")

[<title>The Dormouse's story</title>]

In [51]:
# Combinador Irmão Subsequente ~
soup.select("#link1 ~ .sister")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### Exercício

Crie um seletor CSS para cada item abaixo.
1. Todas as tags b que estão dentro de um p
2. Todas as tags da classe story
3. Tag com id link2 e as suas irmãs subsequentes 